In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [2]:
import os
import sys
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).parent / "src"))

In [3]:
from data_loader import DataLoader
from utils import *
from mcs_function import *
from plotting import *
import config
import pickle
import pmcx
import matplotlib.pyplot as plt
from mbll_functions import *
import pandas as pd
import scipy
import pickle

In [4]:
pmcx.gpuinfo()

=============================   GPU Information  ================================
Device 1 of 1:		NVIDIA GeForce GTX 1050
Compute Capability:	6.1


[{'name': 'NVIDIA GeForce GTX 1050',
  'id': 1,
  'devcount': 1,
  'major': 6,
  'minor': 1,
  'globalmem': 4229824512,
  'constmem': 65536,
  'sharedmem': 49152,
  'regcount': 65536,
  'clock': 1493000,
  'sm': 5,
  'core': 640,
  'autoblock': 64,
  'autothread': 10240,
  'maxgate': 0}]

Global Memory:		4229824512 B
Constant Memory:	65536 B
Shared Memory:		49152 B
Registers:		65536
Clock Speed:		1.49 GHz
Number of SMs:		5
Number of Cores:	640
Auto-thread:		10240
Auto-block:		64


In [5]:
config.gpuid

1

In [6]:
config.simulated_dataset_dir

PosixPath('/media/leon/T7/IDP/dataset/simulated')

Determine tumor parameters via linear least squares from attenuation data.

In [7]:
#tumor_df = pd.read_excel(
#    config.spectra_dir / "tumor_optical_properties.xlsx",
#    sheet_name="HGG_fresh", # high-grade glioma, fresh
#)
tumor_df = pd.read_excel(
    config.spectra_dir / "tumor_optical_properties.xlsx",
    sheet_name="HGG_comparison"
)
tumor_df_wavelength_condition = (tumor_df["Wavelength"] >= 400) & (tumor_df["Wavelength"] <= 1000) 
tumor_absorption_spectrum = tumor_df.loc[tumor_df_wavelength_condition, "mua_Gebhart_frozen_freshly_excised_snapfrozen"].to_numpy() * 10
tumor_scattering_spectrum = tumor_df.loc[tumor_df_wavelength_condition, "mus_fitted_present_fresh_30mins"].to_numpy() * 10
wavelengths_tumor = tumor_df.loc[tumor_df_wavelength_condition, "Wavelength"].to_numpy()

In [8]:
mu_a_tumor = DataLoader.absorption_coefs(
    wavelengths_tumor,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [9]:
res = scipy.optimize.lsq_linear(
    mu_a_tumor,
    tumor_absorption_spectrum,
    bounds=(
        [0, 0, 0, 0, 0, 0],
        [1, 1, 0.1, 0.1, 1, 1]
    )
)
tumor_base_concentrations = res.x
print(tumor_base_concentrations)
print(DataLoader.tissue_parameters["gray matter"][0])

[2.63917345e-02 9.62995787e-29 5.07849077e-02 5.89194355e-28
 1.00000000e+00 2.81937901e-26]
[0.0646 0.0114 0.0064 0.0016 0.73   0.1   ]


The oxCCO value seems much to high. There might be another chromophore that I am not accounting for. I will instead simply use tissue concentrations as for white matter. White matter tissue was reported to have very similar absorption spectra to brain tumor.

In [ ]:
# simulated tissue concentrations
simulation_data = dict()
for tissue_type in ["gray matter", "artery", "vein", "tumor"]: #DataLoader.tissue_parameters.keys():
    base_concentrations = DataLoader.tissue_parameters[tissue_type][0]
    base_scattering_params = DataLoader.tissue_parameters[tissue_type][1]
    std_dev_concentrations_blood_fraction = np.array([0.015, 0.015, 1e-3, 1e-3, 0.015, 0.015])
    sampled_concentrations_blood_fraction = np.random.normal(
        loc=concentrations_to_blood_fraction(base_concentrations),
        scale=std_dev_concentrations_blood_fraction,
        size=(10,6)
    )
    std_dev_scattering = np.array([5, 0.15])
    sampled_scattering_params = np.random.normal(
        loc=base_scattering_params,
        scale=std_dev_scattering,
        size=(10,2)
    )
    # clip values
    sampled_concentrations_blood_fraction = np.clip(sampled_concentrations_blood_fraction, a_min=[[0, 0, 0, 0, 0, 0]], a_max=[[1, 1, np.inf, np.inf, 1, 1]])
    # (#molecules, #spectra)
    sampled_concentrations = blood_fraction_to_concentrations(sampled_concentrations_blood_fraction.T)
    sampled_scattering_params = np.clip(sampled_scattering_params, a_min=[[0, 0]], a_max=[[200, 10]]).T
    # first value is baseline, second should have full perturbation of standard_deviation
    sampled_concentrations = np.column_stack(
        (
         base_concentrations,
         blood_fraction_to_concentrations(base_concentrations + std_dev_concentrations_blood_fraction),
         sampled_concentrations
        )
    )
    sampled_scattering_params = np.column_stack((base_scattering_params, (base_scattering_params + std_dev_scattering), sampled_scattering_params))
    simulation_data[tissue_type] = (sampled_concentrations, sampled_scattering_params)

In [23]:
simulation_data["gray matter"][0].shape

(6, 12)

In [21]:
simulation_data["tumor"][0]

array([[3.23000000e-02, 2.27700000e-03, 8.11328027e-02, 4.53423199e-02,
        5.67067660e-02, 5.94951082e-02, 3.71946965e-02, 0.00000000e+00,
        4.68674372e-02, 7.26746317e-02, 7.09336001e-03, 3.58486562e-02],
       [5.70000000e-03, 1.07723000e-01, 1.22538644e-02, 7.19931674e-03,
        1.22207515e-02, 1.25605463e-02, 6.33177545e-03, 0.00000000e+00,
        8.36757770e-03, 1.37233241e-02, 1.34924479e-03, 5.89923952e-03],
       [3.20000000e-03, 4.20000000e-03, 3.09638521e-03, 3.62876329e-03,
        3.65762935e-03, 2.97536051e-03, 4.03934032e-03, 3.39966473e-03,
        3.01722846e-03, 2.85640935e-03, 2.60114620e-03, 5.13236749e-03],
       [8.00000000e-04, 1.80000000e-03, 9.59151367e-04, 6.22603811e-04,
        1.55420296e-03, 3.40901211e-05, 7.32427604e-04, 8.94125316e-04,
        7.44636029e-04, 5.66474487e-04, 2.53531195e-04, 0.00000000e+00],
       [3.65000000e-01, 3.80000000e-01, 3.72026453e-01, 3.71080532e-01,
        3.63153230e-01, 3.91893871e-01, 3.61726566e-01, 3.41

MCX Configuration

In [ ]:
# empty log file
open(config.simulated_dataset_dir / "log.txt", "w").close()
vol = np.ones((60, 60, 120), dtype="uint8")

cfg = {
    "nphoton": 1e7,
    "maxdetphoton": 1e7,
    "vol": vol,
    "tstart":0,
    "tend":5e-7,
    "tstep":5e-7,
    "autopilot": 1,
    "gpuid": config.gpuid,
    "bc": "ccrcca",#001000",
    "isspecular": 1,
    "srcdir": [0,0,1],
    "srctype": "planar",
    "srcpos": [0, 0, 0],
    "srcparam1": [vol.shape[0], 0, 0, 0], # 3D position of vertex, fourth coordinate is irrelevant
    "srcparam2": [0, vol.shape[1], 0, 0],
    "issrcfrom0": 1,
    #"savedetflag": "dpx",
    "flog": str(config.simulated_dataset_dir) + "/log.txt",
    "isnormalized": 0,
    "issaveref": 1
}

# necessary to accumulate weights
cfg["vol"][:, :, 0] = 0

In [ ]:
wavelengths = np.arange(520, 910, 10)
mu_a_matrix = DataLoader.absorption_coefs(
    wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [ ]:
for tissue_type in simulation_data.keys():
    tissue_spectra = []
    g, refractive_idx = DataLoader.tissue_parameters[tissue_type][-2:]
    for concentrations, scattering_params in zip(*simulation_data[tissue_type][:2]):
        mu_a_vals = mu_a_matrix @ concentrations
        mu_s_red_vals = scattering_params[0] * (wavelengths / 500) ** (-scattering_params[1])
        mu_a_vals /= 10
        mu_s_red_vals /= 10
        spectrum = []
        for mu_a, mu_s_red in zip(mu_a_vals, mu_s_red_vals):
            cfg["prop"] = np.array([
                [0, 0, 1, 1],
                [mu_a, mu_s_red / (1-g), g, refractive_idx]
            ])
            res = pmcx.mcxlab(cfg)
            dref = np.sum(res["dref"][:, :, 0, :]) / cfg["nphoton"]
            spectrum.append(-np.log(dref))
        tissue_spectra.append(spectrum)
    # notice the .T => store as (wavelengths, spectra)
    simulation_data[tissue_type] = simulation_data[tissue_type][:2] + (np.array(tissue_spectra).T,)

Save data to file...

In [ ]:
with open(config.simulated_dataset_dir / "simulation_data.pickle", "wb") as f:
    pickle.dump(simulation_data, f)

In [ ]:
plot_spectra_slider(simulation_data["gray_matter"][-1], wavelengths)